# Edgar Filings - Shares Outstanding

This notebook shows how to use Algoseek's Edgar Filings dataset to get the shares outstanding for a company. The shares outstanding is the total number of shares of a company's stock that are publicly owned and available for trading. It is an important metric for investors to understand the company's market capitalization and ownership structure. The information is extracted from the 10-Q and 10-K filings of the company. The dataset provides the shares outstanding for each filing date.

NOTE: This dataset provides data as-is from the SEC filings. In the case of shares outstanding data this means that the data is not adjusted for any corporate actions such as stock splits, reverse splits, etc.



In [15]:
import os
import pandas as pd
from IPython.display import HTML, display
import clickhouse_connect
from dotenv import load_dotenv

load_dotenv()

client = clickhouse_connect.get_client(
    host=os.environ["CLICKHOUSE_HOST"],
    port=int(os.environ["CLICKHOUSE_PORT"]),
    user=os.environ["CLICKHOUSE_USER"],
    password=os.environ["CLICKHOUSE_PASSWORD"],
    database=os.environ["CLICKHOUSE_DATABASE"],
)

## Display a time series of a single concept

For this example, we display the time series of the concept `CommonStockSharesOutstanding`, available in the `10-K` and `10-Q `reports, for the ticker: `AMZN`.

### The query


In [21]:
database = os.environ["CLICKHOUSE_DATABASE"]
ticker = "AMZN"

query = f"""
    SELECT DISTINCT CONCAT(f.FiscalYear, ' - ', f.FiscalPeriod) AS FiscalYearPeriod, f.Value
    FROM {database}.financials f
    WHERE f.EntityId IN (SELECT e.EntityId FROM {database}.entities e WHERE e.Ticker in ('{ticker}')) 
      AND f.ConceptName = 'CommonStockSharesOutstanding' 
    ORDER BY FiscalYearPeriod
"""

### Response

We query the database and display the results.

As you can see, the values reported reflect the 20:1 2022 split. However, since this is as-reported data, it does not contain any adjustment factors. We are working on obtaining the adjustment factors in the Edgar Filings Dataset, but for now, we have them available as a separate dataset.


In [22]:
result = client.query(query)

# Convert the result to a pandas DataFrame
df = pd.DataFrame(result.result_rows, columns=result.column_names)

# Display the DataFrame
display(HTML(df.to_html(notebook=True, index=False)))

FiscalYearPeriod,Value
2008 - Y,428000000
2009 - 2Q,432000000
2009 - 3Q,433000000
2009 - Y,444000000
2010 - 1Q,446000000
2010 - 2Q,448000000
2010 - 3Q,449000000
2010 - Y,451000000
2011 - 1Q,452000000
2011 - 2Q,454000000
